In [1]:
from dotenv import load_dotenv
import os
import pygsheets
import requests
import pandas as pd
from tqdm import tqdm

from pangres import upsert
from sqlalchemy import text, create_engine

load_dotenv()

True

In [4]:
db_username=os.environ.get("db_username")
db_password=os.environ.get("db_password")
db_host=os.environ.get("db_host")
db_port=os.environ.get("db_port")
# db_name=os.environ.get("db_name")
db_name = 'lol_analytics'


def create_db_connection_string(db_username: str, db_password: str, db_host: str, db_port: int, db_name: str):
    connection_url = f"postgresql+psycopg2://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}"
    return connection_url

## Functions

In [5]:
def get_puuid(gameName, tagline, api_key):
    link = f'https://americas.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{username}/{tagline}?api_key={api_key}'

    response = requests.get(link)
    if response.status_code == 200:
        return response.json()['puuid']
    else:
        print(f"Error: {response.status_code}")
    return None

In [6]:
def account_info_by_puuid(puuid, api_key):
    link = f'https://americas.api.riotgames.com/riot/account/v1/accounts/by-puuid/{puuid}?api_key={api_key}'

    response = requests.get(link)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
    return None

In [7]:
def retrieve_match_ids(puuid, api_key, type='ranked'):
    link = f'https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?type={type}&start=0&count=100&api_key={api_key}'

    response = requests.get(link)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
    return None

In [8]:
def retrieve_match_data(match_id, api_key):
    link = f'https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={api_key}'
    response = requests.get(link)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
    return response.json()

In [ ]:
def process_match_json(match_id, puuid, api_key):
    game = retrieve_match_data(match_id, api_key)

    metadata = game['metadata']
    data_version = metadata['dataVersion']
    match_id = metadata['matchId']
    participants_puuid = metadata['participants']

    match_info = game['info']
    match_duration = match_info['gameDuration'] # 
    match_mode = match_info['gameMode']
    match_type = match_info['gameType']
    match_version = match_info['gameVersion']
    match_map_id = match_info['mapId']
    match_queue_id = match_info['queueId']
    participant_data = match_info['participants']


    main_player = participant_data[participants_puuid.index(puuid)]
    champion_level = main_player['champLevel']
    champion_experience = main_player['champExperience']
    champion_name = main_player['championName']
    champion_id = main_player['championId']
    deaths = main_player['deaths']
    kills = main_player['kills']
    assists = main_player['assists']
    gold_earned = main_player['goldEarned']
    gold_spent = main_player['goldSpent']
    individual_position = main_player['individualPosition']
    item_0 = main_player['item0']
    item_1 = main_player['item1']
    item_2 = main_player['item2']
    item_3 = main_player['item3']
    item_4 = main_player['item4']
    item_5 = main_player['item5']
    item_6 = main_player['item6']
    lane = main_player['lane']
    neutral_minions_killed = main_player['neutralMinionsKilled']
    objectives_stolen = main_player['objectivesStolen']
    participant_id = main_player['participantId']
    riot_id_game_name = main_player['riotIdGameName']
    riot_id_tagline = main_player['riotIdTagline']
    role = main_player['role']
    total_damage_dealt = main_player['totalDamageDealt']
    total_damage_dealt_to_champions = main_player['totalDamageDealtToChampions']
    total_damage_shielded_on_teammates = main_player['totalDamageShieldedOnTeammates']
    total_damage_taken = main_player['totalDamageTaken']
    totalHealsOnTeammates = main_player['totalHealsOnTeammates']
    total_minions_killed = main_player['totalMinionsKilled']
    total_time_cc_dealt = main_player['totalTimeCCDealt']
    team_id = main_player['teamId']
    team_position = main_player['teamPosition']
    turret_kills = main_player['turretKills']
    vision_score = main_player['visionScore']
    vision_wards_bought = main_player['visionWardsBoughtInGame']
    wards_placed = main_player['wardsPlaced']
    wards_killed = main_player['wardsKilled']
    win = main_player['win']

    # perks
    main_player_perks = main_player['perks']
    stats_perks = main_player_perks['statPerks']
    perk_defense = stats_perks['defense']
    perk_flex = stats_perks['flex']
    perk_offense = stats_perks['offense']

    perk_style_selection = main_player_perks['styles']

    primary_style = perk_style_selection[0]
    primary_style_selections = primary_style['selections']
    keystone = primary_style_selections[0]['perk']
    primary_style_selection_1 = primary_style_selections[1]['perk']
    primary_style_selection_2 = primary_style_selections[2]['perk']
    primary_style_selection_3 = primary_style_selections[3]['perk']

    secondary_style = perk_style_selection[1]
    secondary_style_selections = secondary_style['selections']
    secondary_style_selection_1 = secondary_style_selections[0]['perk']
    secondary_style_selection_2 = secondary_style_selections[1]['perk']


    for team in match_info['teams']:
        team_id = team['teamId']
        team_win = team['win']
        bans = team['bans']

        if bans:
            for ban in bans:
                champion_id = ban['championId']
                # champion_name = ban['championName']


        objectives = team['objectives']
        if 'atakhan' in objectives:
            atakhan = objectives['atakhan']
        else:
            atakhan = 0
        dragon = objectives['dragon']
        inhibitor = objectives['inhibitor']
        rift_herald = objectives['riftHerald']
        tower = objectives['tower']
        baron = objectives['baron']
        horde = objectives['horde']
        champion = objectives['champion']


    match_dataframe = pd.DataFrame(
        {
            'data_version': [data_version],
            'match_id': [match_id],
            'puuid': [puuid],
            'participants_puuid': [participants_puuid],
            'match_mode': [match_mode],
            'match_type': [match_type],
            'match_duration': [match_duration],
            'match_version': [match_version],
            'match_map_id': [match_map_id],
            'match_queue_id': [match_queue_id],
            'champion_level': [champion_level],
            'champion_experience': [champion_experience],
            'champion_name': [champion_name],
            'champion_id': [champion_id],
            'deaths': [deaths],
            'kills': [kills],
            'assists': [assists],
            'gold_earned': [gold_earned],
            'gold_spent': [gold_spent],
            'individual_position': [individual_position],
            'item_0': [item_0],
            'item_1': [item_1],
            'item_2': [item_2],
            'item_3': [item_3],
            'item_4': [item_4],
            'item_5': [item_5],
            'item_6': [item_6],
            'lane': [lane],
            'neutral_minions_killed': [neutral_minions_killed],
            'objectives_stolen': [objectives_stolen],
            'participant_id': [participant_id],
            'riot_id_game_name': [riot_id_game_name],
            'riot_id_tagline': [riot_id_tagline],
            'role': [role],
            'total_damage_dealt': [total_damage_dealt],
            'total_damage_dealt_to_champions': [total_damage_dealt_to_champions],
            'total_damage_shielded_on_teammates': [total_damage_shielded_on_teammates],
            'total_damage_taken': [total_damage_taken],
            'totalHealsOnTeammates': [totalHealsOnTeammates],
            'total_minions_killed': [total_minions_killed],
            'total_time_cc_dealt': [total_time_cc_dealt],
            'team_id': [team_id],
            'team_position': [team_position],
            'turret_kills': [turret_kills],
            'vision_score': [vision_score],
            'vision_wards_bought': [vision_wards_bought],
            'wards_placed': [wards_placed],
            'wards_killed': [wards_killed],
            'win': [win],
            'keystone': [keystone],
            'primary_style_selection_1': [primary_style_selection_1],
            'primary_style_selection_2': [primary_style_selection_2],
            'primary_style_selection_3': [primary_style_selection_3],
            'secondary_style_selection_1': [secondary_style_selection_1],
            'secondary_style_selection_2': [secondary_style_selection_2],
            'perk_defense': [perk_defense],
            'perk_flex': [perk_flex],
            'perk_offense': [perk_offense],
            'team_id': [team_id],
            'team_win': [team_win],
            'atakhan': [atakhan],
            'dragon': [dragon],
            'inhibitor': [inhibitor],
            'rift_herald': [rift_herald],
            'tower': [tower],
            'baron': [baron],
            'horde': [horde],
            'champion': [champion],
        }
    )
    return match_dataframe

In [116]:
username = 'mazsu'
tagline = 'mas'

api_key = os.getenv("riot_api_key")
puuid = get_puuid(username, tagline, api_key)

account_info_by_puuid(puuid, api_key)
match_ids = retrieve_match_ids(puuid, api_key)

dataframe_list = []

for match_id in tqdm(match_ids):
    match_dataframe = process_match_json(match_id, puuid, api_key)
    dataframe_list.append(match_dataframe)



 97%|█████████▋| 97/100 [00:25<00:00,  3.77it/s]


Error: 429


KeyError: 'metadata'

In [117]:
all_matches_dataframe = pd.concat(dataframe_list)

In [118]:
def retrieve_perk_info():
    perks = requests.get('https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/perks.json').json()
    perkstyles = requests.get('https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/perkstyles.json').json()
    
    perk_dict = {}
    for i in perkstyles['styles']:
        perk_dict[i['id']] = i['name']

    for i in perks:
        perk_dict[i['id']] = i['name']
    return perk_dict

In [ ]:
perk_dict = retrieve_perk_info()
perk_columns = ['keystone', 'primary_style_selection_1', 'primary_style_selection_2', 
                'primary_style_selection_3', 'secondary_style_selection_1', 
                'secondary_style_selection_2', 'perk_defense', 'perk_flex', 'perk_offense']

all_matches_dataframe[perk_columns] = all_matches_dataframe[perk_columns].replace(perk_dict)

In [ ]:
def retrieve_items_info():
    items = requests.get('https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/items.json').json()
    
    items_dict = {}
    for i in items:
        items_dict[i['id']] = i['name']
    return items_dict

In [ ]:
items_dict = retrieve_items_info()
item_columns = ['item_0', 'item_1', 'item_2', 'item_3', 'item_4', 'item_5', 'item_6']
all_matches_dataframe[item_columns] = all_matches_dataframe[item_columns].replace(items_dict)

NameError: name 'all_matches_dataframe' is not defined

In [121]:
all_matches_dataframe['username'] = [username]*len(all_matches_dataframe)
all_matches_dataframe['tagline'] = [tagline]*len(all_matches_dataframe)

In [122]:
all_matches_dataframe['uuid'] = all_matches_dataframe['match_id'] + '_' + all_matches_dataframe['puuid']
all_matches_dataframe = all_matches_dataframe.set_index('uuid')

In [123]:
connection_url = create_db_connection_string(db_username, db_password, db_host, db_port, db_name)
db_engine = create_engine(connection_url, pool_recycle=3600)
connection = db_engine.connect()
upsert(con=connection, df=all_matches_dataframe, schema='soloq', table_name='player_matches', create_schema=True, if_row_exists='update')

In [48]:
connection.commit()

## Simple Analytics

In [44]:
connection_url = create_db_connection_string(db_username, db_password, db_host, db_port, db_name)
db_engine = create_engine(connection_url, pool_recycle=3600)
connection = db_engine.connect()

In [49]:
with db_engine.connect() as connection:
    df = pd.read_sql(text("SELECT * FROM soloq.player_matches"), connection)


In [34]:
df['lane'].unique()
df[df['username'] == 'RoboBoto']['team_position'].value_counts()

team_position
UTILITY    81
BOTTOM     15
JUNGLE      1
Name: count, dtype: int64

In [157]:
main_lane_games = df[(df['team_position'] == 'UTILITY') & (df['username'] == 'mazsu')]
main_lane_games['champion_name'].value_counts()

champion_name
Pyke         4
TahmKench    4
Poppy        3
Neeko        3
Braum        3
Pantheon     2
Morgana      1
Rell         1
Bard         1
Velkoz       1
Brand        1
Ornn         1
Rakan        1
Karma        1
Alistar      1
Thresh       1
Mel          1
Xerath       1
Name: count, dtype: int64

In [158]:
mains = main_lane_games['champion_name'].value_counts()[:3].reset_index()['champion_name'].tolist()

In [159]:
mains_only = main_lane_games[main_lane_games['champion_name'].isin(mains)]
mains_only['champion_name'].value_counts()

champion_name
TahmKench    4
Pyke         4
Poppy        3
Name: count, dtype: int64

In [160]:
dragon = mains_only.set_index('champion_name')['dragon']
dragon_list = []
for i, row in enumerate(dragon):
    dragon_list.append(row)

dragon_df = pd.DataFrame(dragon_list)

In [161]:
dragon = dragon.reset_index()
dragon['dragon'] = dragon_df['kills']
dragon['first_kill'] = dragon_df['first']

In [162]:
dragon.groupby('champion_name').mean()

,dragon,first_kill
champion_name,,
Poppy,3.00,0.666667
Pyke,1.50,0.250000
TahmKench,1.75,0.500000


In [163]:
summarized_stats = mains_only.groupby('champion_name')[['match_duration', 'champion_level','turret_kills', 'total_damage_dealt_to_champions', 'wards_killed','wards_placed', 'gold_earned', 'total_minions_killed', 'neutral_minions_killed', 'kills', 'deaths','assists', 'win']].mean()

summarized_stats['match_duration_in_minutes'] = summarized_stats['match_duration']/60
summarized_stats['cs_per_minute'] = (summarized_stats['total_minions_killed']+summarized_stats['neutral_minions_killed'])/summarized_stats['match_duration_in_minutes']
summarized_stats

,match_duration,champion_level,turret_kills,total_damage_dealt_to_champions,wards_killed,wards_placed,gold_earned,total_minions_killed,neutral_minions_killed,kills,deaths,assists,win,match_duration_in_minutes,cs_per_minute
champion_name,,,,,,,,,,,,,,,
Poppy,1843.00,14.333333,0.00,17674.666667,8.666667,31.333333,10529.666667,27.333333,0.666667,6.00,7.00,14.666667,0.666667,30.716667,0.911557
Pyke,1657.75,12.750000,0.00,16733.500000,13.500000,28.750000,10198.000000,36.000000,0.000000,8.75,7.25,9.750000,0.250000,27.629167,1.302971
TahmKench,1552.00,11.250000,0.25,9325.750000,4.500000,23.000000,7135.500000,23.500000,0.250000,1.50,5.50,10.500000,0.750000,25.866667,0.918170


In [ ]:
# need to create 2 databases? maybe retrieve as much data as possible for challenger ladder?
# how would we update something so frequent consideirng that to compare stats we would need constant updating 
# maybe user windows scheduler to update the live db?